# Client Partner Implementation Outline


In [206]:
import re
import json, requests, xml
import pandas as pd
# import ipywidgets as widgets

from xml.dom import minidom
# from IPython.display import Image
from xml.etree import ElementTree
from IPython.display import HTML
from IPython.display import Markdown as md

In [207]:
def get_api_request(template, os_querystring):
  # Fill (URL) template with OpenSearch parameter values provided in os_querystring and return as short HTTP URL without empty parameters.
  
  # print("URL template: " + template)
  
  # Limitation: the OSDD may use a default namespace for OpenSearch instead of using "os".
  # We make a simple correction here allowing to use OpenSearch queryables without namespace in requests.
  # A more generic solution to obtain namespaces from the OSDD and compare them with user supplied namespaces is future work.
  
  OS_NAMESPACE = 'os:'
      
  # perform substitutions in template
  for p in os_querystring:
      # print("  .. replacing:", p, "by", os_querystring[p])
      # template = re.sub('\{'+p+'.*?\}', os_querystring[p] , template)
      result = re.subn('\{'+p+'.*?\}', os_querystring[p] , template)
      n = result[1]
      template = result[0]
      if (n<1):
          if (':' in p):
                print("ERROR: parameter " + p + " not found in template.")
          else:
                # try with explicit namespace
                result = re.subn('\{'+OS_NAMESPACE+p+'.*?\}', os_querystring[p] , template)
                n = result[1]
                template = result[0]
                if (n<1):
                    print("ERROR: parameter " + OS_NAMESPACE+p + " not found in template.")   
      
      # print("- intermediate new template:" + template)
      
  # remove empty search parameters
  template=re.sub('&?[a-zA-Z]*=\{.*?\}', '' , template)
  
  # remove remaining empty search parameters which did not have an HTTP query parameter attached (e.g. /{time:end}).
  template=re.sub('.?\{.*?\}', '' , template)
  
  # print("API request: " + template)
            
  return (template)

## Client Search Scenario

All Connected Data Asset clients are, at their core, just conventional OpenSearch clients with a few extensions and conventions to make the CEOS OpenSearch protocol useful in the WGISS environment. The IDN identifier of the collection of interest is a mandatory element in CEOS OpenSearch request. Clients could retrieve the collection ID from the IDN OpenSearch Description Document (OSDD).  

This chapter will give brief steps about how to retrieve an IDN collection ID and how to interact with the granule search server for inventory search. The corresponding details are elaborated in the Use Case chapters. 

Data providers will have registered the metadata of their archived collections into the IDN. The client can query the IDN to retrieve the IDN collection ID for a desired collection of interest and, based on that collection ID and other spatial-temporal query conditions, build a valid CEOS OpenSearch query. The following steps describe the client search scenario starting with IDN OpenSearch. 

**Step 1**  
>  Obtain the IDN OpenSearch OSDD to formulate a valid IDN OpenSearch request.

In [208]:
URL_OSDD = "https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc"

In [209]:
md("The OpenSearch Description Document is accessible at the fixed location [{url}]({url}) and contains the URL template to be used for collection search.".format(url=URL_OSDD))

The OpenSearch Description Document is accessible at the fixed location [https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc](https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc) and contains the URL template to be used for collection search.

In [210]:
response = requests.get( URL_OSDD )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><os:OpenSearchDescription xmlns:atom="http://www.w3.org/2005/Atom" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:referrer="http://www.opensearch.org/Specifications/OpenSearch/Extensions/Referrer/1.0" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">  
    <os:ShortName>CMR Collections</os:ShortName>  
    <os:Description>NASA CMR Collection search using geo, time and parameter extensions</os:Description>  
    <os:Contact>echodev@echo.nasa.gov</os:Contact>  
    <os:Url params:method="GET" rel="collection" template="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&amp;instrument={echo:instrument?}&amp;satellite={eo:platform?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;hasGranules={echo:hasGranules?}&amp;isCwic={echo:isCwic?}&amp;isGeoss={echo:isGeoss?}&amp;isCeos={echo:isCeos?}&amp;isEosdis={echo:isEosdis?}&amp;isFedeo={echo:isFedeo?}&amp;provider={echo:provider?}&amp;clientId=ceosOpenSearchDoc" type="application/atom+xml">    
        <params:Parameter minimum="0" name="keyword" title="Inventory with terms expressed by these search terms" uiDisplay="Search terms" value="{os:searchTerms}">      
            <atom:link href="http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html" rel="profile" title="This parameter follows the elastic search free text search implementations"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="instrument" title="Inventory associated with a satellite instrument expressed by this short name" uiDisplay="Instrument" value="{echo:instrument}"/>    
        <params:Parameter minimum="0" name="satellite" title="Inventory associated with a Satellite/platform expressed by this short name" uiDisplay="Satellite" value="{eo:platform}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier" value="{geo:uid}"/>    
        <params:Parameter name="hasGranules" title="Inventory with granules" uiDisplay="Has granules" value="{echo:hasGranules}">      
            <params:Option label="Yes" value="true"/>      
            <params:Option label="No" value="false"/>      
        </params:Parameter>    
        <params:Parameter name="isCwic" title="Inventory related to CWIC" uiDisplay="CWIC collection" value="{echo:isCwic}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isGeoss" title="Inventory related to GEOSS" uiDisplay="GEOSS collection" value="{echo:isGeoss}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isCeos" title="Inventory related to CEOS" uiDisplay="CEOS collection" value="{echo:isCeos}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isEosdis" title="Inventory related to EOSDIS" uiDisplay="EOSDIS collection" value="{echo:isEosdis}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isFedeo" title="Inventory related to FedEO" uiDisplay="FedEO collection" value="{echo:isFedeo}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="provider" title="Inventory associated with a provider" uiDisplay="Provider" value="{echo:provider}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  
    <os:Url params:method="GET" rel="collection" template="https://cmr.earthdata.nasa.gov/opensearch/collections.html?keyword={os:searchTerms?}&amp;instrument={echo:instrument?}&amp;satellite={eo:platform?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;hasGranules={echo:hasGranules?}&amp;isCwic={echo:isCwic?}&amp;isGeoss={echo:isGeoss?}&amp;isCeos={echo:isCeos?}&amp;isEosdis={echo:isEosdis?}&amp;isFedeo={echo:isFedeo?}&amp;provider={echo:provider?}&amp;clientId=ceosOpenSearchDoc" type="text/html">    
        <params:Parameter minimum="0" name="keyword" title="Inventory with terms expressed by these search terms" uiDisplay="Search terms" value="{os:searchTerms}">      
            <atom:link href="http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html" rel="profile" title="This parameter follows the elastic search free text search implementations"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="instrument" title="Inventory associated with a satellite instrument expressed by this short name" uiDisplay="Instrument" value="{echo:instrument}"/>    
        <params:Parameter minimum="0" name="satellite" title="Inventory associated with a Satellite/platform expressed by this short name" uiDisplay="Satellite" value="{eo:platform}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier" value="{geo:uid}"/>    
        <params:Parameter name="hasGranules" title="Inventory with granules" uiDisplay="Has granules" value="{echo:hasGranules}">      
            <params:Option label="Yes" value="true"/>      
            <params:Option label="No" value="false"/>      
        </params:Parameter>    
        <params:Parameter name="isCwic" title="Inventory related to CWIC" uiDisplay="CWIC collection" value="{echo:isCwic}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isGeoss" title="Inventory related to GEOSS" uiDisplay="GEOSS collection" value="{echo:isGeoss}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isCeos" title="Inventory related to CEOS" uiDisplay="CEOS collection" value="{echo:isCeos}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isEosdis" title="Inventory related to EOSDIS" uiDisplay="EOSDIS collection" value="{echo:isEosdis}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isFedeo" title="Inventory related to FedEO" uiDisplay="FedEO collection" value="{echo:isFedeo}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="provider" title="Inventory associated with a provider" uiDisplay="Provider" value="{echo:provider}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  
    <os:Query role="example" searchTerms="Amazon River Basin Precipitation, 1972-1992" title="Sample search"/>  
    <os:Attribution>NASA CMR</os:Attribution>  
    <os:SyndicationRight>open</os:SyndicationRight>  
    <os:Tags>CMR NASA CWIC CEOS-OS-BP-V1.1/L3 ESIP OGC collection pageOffset=1 indexOffset=0</os:Tags>  
</os:OpenSearchDescription>
```


The OSDD may contain URL templates for multiple values for `type` (media types) and multiple values for `rel` (relations).  The `rel` values have the following meaning (if present) as defined in [[RD1]](#RD1). :

| **rel** |  **description**  | 
| ----- | ----- |
| collection |  URL template to be used for collection search.  |
| results |  URL template to be used for granule search (default).  |
| service |  URL template to be used for service or application search.  |

The current OSDD contains the following URL templates:

In [211]:

root = ElementTree.fromstring(response.text)

list = pd.DataFrame(columns=['rel', 'type'])
ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
for r in root.findall('os:Url', ns):  
    rel = ''
    mtype = ''
    try:
        rel   = r.attrib['rel']
        mtype = r.attrib['type']
    except:
        pass
    df = pd.DataFrame({ 'rel': rel, 'type': mtype }, index = [0])
    list = pd.concat([list, df], ignore_index=True)  
  
list     

,rel,type
0,collection,application/atom+xml
1,collection,text/html


In [212]:
# find URL template for collection search
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
collection_url_atom = root.find('os:Url[@rel="collection"][@type="application/atom+xml"]', ns)

collection_template = collection_url_atom.attrib['template']
collection_template

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc'

**Step 2**  
>  Search collections of interest through IDN OpenSearch with proper request parameters (e.g. spatial footprint, temporal extent and keyword). 

In [213]:
request_url = get_api_request(collection_template, {'count': '10', 'searchTerms': 'Landsat*'})
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&numberOfResults=10&clientId=ceosOpenSearchDoc'

In [214]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-12-06T15:24:51.581Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; Landsat*</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=66" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=2" hreflang="en-US" rel="next" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query os:searchTerms="Landsat*" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>659</os:totalResults>   
     <os:itemsPerPage>10</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1965336923-ESA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>FEDEO</consortium>      
          <title type="text">Landsat 8 OLI-TIRS European Coverage</title>      
          <summary type="text">The European coverage of Landsat 8 data since the beginning of the mission until December 2014 is available on the Landsat 8 portal. Data shown on the portal: portal are reprocessed (Tier 1 and Tier 2) products acquired at ESA receiving stations (Neustrelitz, Matera and Kiruna visibility masks).: You can find the most recent data (from 2015 onwards) in the Landsat8 European Coverage Collection 2 Level 1 and Level2; as soon as the new collection is completely uploaded and the entire mission is covered, the former dissemination system for Collection 1 dataset will be dismissed. Landsat 8 level 1 products combine data from the two Landsat instruments, OLI and TIRS. The level 1 products generated can be either L1T or L1gT (30m resolution, 170 km x 185 km scene size): • L1T - Orthorectified products Level 1T data products consist of radiometrically corrected image data derived from L0 data scaled to at-aperture spectral radiance or reflectance that are resampled for registration to a cartographic projection (referenced to the WGS84, G873 or current version). The L1T product is orthorectified, and corrected for terrain relief. The geometric corrections use observatory ephemeris data and ground control points; DEM data is used to correct for terrain relief. • L1gT - Geometrically corrected products L1gT data products consist of L0 product data with systematic radiometric, geometric and terrain corrections applied and resampled for registration to a cartographic projection, referenced to the WGS84, G873, or current version. L1gT data products assume the use of on-board positional information or definitive ephemeris, as well as the use of controlled elevation data to correct for parallax errors. In case of insufficient GCP, caused by extensive snow/cloud cover, which makes the orthorectification impossible, products of this level are produced instead of L1T.</summary>      
          <updated/>      
          <link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=LANDSAT.OLI-TIRS.L1T_L1GT" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=LANDSAT.OLI-TIRS.L1T_L1GT&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT.OLI-TIRS.L1T_L1GT_NA" hreflang="en-US" rel="enclosure" title="LANDSAT.OLI-TIRS.L1T_L1GT" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1965336923-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1965336923-ESA</dc:identifier>      
          <dc:date>2014-12-31T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8 OLI-TIRS European Coverage</echo:datasetId>      
          <echo:shortName>LANDSAT.OLI-TIRS.L1T_L1GT</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:entryId>LANDSAT.OLI-TIRS.L1T_L1GT_NA</echo:entryId>      
          <echo:dataCenter>ESA</echo:dataCenter>      
          <echo:organization>ESA/ESRIN</echo:organization>      
          <echo:processingLevelId>1</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-35 -20 75 35</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine/>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>int.esa.fedeo</echo:tagKey>         
          </echo:tag>      
          <echo:is_fedeo>true</echo:is_fedeo>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C2241716601-ESA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>FEDEO</consortium>      
          <title type="text">Landsat 8 Collection 2 European Coverage</title>      
          <summary type="text">This dataset contains the European Coverage of Landsat 8 Collection 2 data, both Level 1 and Level 2, since the beginning of the mission. Landsat 8 Collection 2 is the result of reprocessing effort on the archive and on fresh products with significant improvement with respect to Collection 1 on data quality, obtained by means of advancements in data processing, algorithm development. The primary characteristic is a relevant improvement in the absolute geolocation accuracy (now re-baselined to the European Space Agency Copernicus Sentinel-2 Global Reference Image, GRI) but includes also updated digital elevation modelling sources, improved Radiometric Calibration (even correction for the TIRS striping effect), enhanced Quality Assessment Bands, updated and consistent metadata files, usage of Cloud Optimized Georeferenced (COG) Tagged Image File Format.  Landsat 8 level 1 products combine data from the 2 Landsat instruments, OLI and TIRS. The level 1 products generated can be either L1TP or L1GT: • L1TP - Level 1 Precision Terrain (Corrected) (L1T) products: Radiometrically calibrated and orthorectified using ground control points (GCPs) and digital elevation model (DEM) data to correct for relief displacement. The highest quality Level-1 products suitable for pixel-level time series analysis. GCPs used for L1TP correction are derived from the Global Land Survey 2000 (GLS2000) data set. • L1GT - Level 1 Systematic Terrain (Corrected) (L1GT) products: L1GT data products consist of L0 product data with systematic radiometric, geometric and terrain corrections applied and resampled for registration to a cartographic projection, referenced to the WGS84, G873, or current version. The dissemination server contains three different classes of Level1 products • Real Time (RT): Newly acquired Landsat 8 OLI/TIRS data are processed upon downlink but use an initial TIRS line-of-sight model parameters; the data is made available in less than 12 hours (4-6 hours typically). Once the data have been reprocessed with the refined TIRS parameters, the products are transitioned to either Tier 1 or Tier 2 and removed from the Real-Time tier (in 14-16 days). • Tier 1 (T1): Landsat scenes with the highest available data quality are placed into Tier 1 and are considered suitable for time-series analysis. Tier 1 includes Level-1 Precision and Terrain (L1TP) corrected data that have well-characterized radiometry and are inter-calibrated across the different Landsat instruments. The georegistration of Tier 1 scenes is consistent and within prescribed image-to-image tolerances of ≦ 12-meter radial root mean square error (RMSE). • Tier 2 (T2): Landsat scenes not meeting Tier 1 criteria during processing are assigned to Tier 2. Tier 2 scenes adhere to the same radiometric standard as Tier 1 scenes, but do not meet the Tier 1 geometry specification due to less accurate orbital information (specific to older Landsat sensors), significant cloud cover, insufficient ground control, or other factors. This includes Systematic Terrain (L1GT) and Systematic (L1GS) processed data.  Landsat 8 level 2 products are generated from L1GT and L1TP Level 1 products that meet the &lt;76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. The data are available a couple of days after the Level1 T1/T2. The level 2 products generated can be L2SP or L2SR: • L2SP - Level 2 Science Products (L2SP) products: include Surface Reflectance (SR), Surface Temperature (ST), ST intermediate bands, an angle coefficients file, and Quality Assessment (QA) Bands. • L2SR - Level 2 Surface Reflectance (L2SR) products: include Surface Reflectance (SR), an angle coefficients file, and Quality Assessment (QA) Bands; it is generated if ST could not be generated  Two different categories of Level 1 products are offered: LC with Optical, Thermal and Quality Map images, LO with Optical and Quality Map images (Thermal not available). For the Level 2 data, only LC combined products are generated</summary>      
          <updated/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=Landsat8.Collection2.European.Coverage" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=Landsat8.Collection2.European.Coverage" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=Landsat8.Collection2.European.Coverage&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?Landsat8.Collection2.European.Coverage_NA" hreflang="en-US" rel="enclosure" title="Landsat8.Collection2.European.Coverage" type="text/html"/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=Landsat8.Collection2.European.Coverage" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C2241716601-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C2241716601-ESA</dc:identifier>      
          <dc:date>2015-01-01T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8 Collection 2 European Coverage</echo:datasetId>      
          <echo:shortName>Landsat8.Collection2.European.Coverage</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:entryId>Landsat8.Collection2.European.Coverage_NA</echo:entryId>      
          <echo:dataCenter>ESA</echo:dataCenter>      
          <echo:organization>ESA/ESRIN</echo:organization>      
          <echo:processingLevelId>1</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-35 -20 75 35</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1214584349-SCIOPS</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <title type="text">LANDSAT MSS Data Received at the NASDA Station in Japan</title>      
          <summary type="text">The LANDSAT Multispectral Scanner (MSS), a mechanical scanning
      radiometer, operates in 4 channels of the electro-magnetic spectrum,
      from 0.55 to 1.00 micrometers.
      
      Data collected by the Earth Observation Center, NASDA, cover a
      circular area about 5000 km diameter.  The Earth Observation Center
      receives MSS data weekdays and every other Saturday. It performs both
      radiometric and geometric correction and distributes products in the
      form of magnetic tape and imagery.</summary>      
          <updated/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT-MSS-NASDA" hreflang="en-US" rel="enclosure" title="LANDSAT-MSS-NASDA" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1214584349-SCIOPS.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1214584349-SCIOPS</dc:identifier>      
          <dc:date>1979-01-29T00:00:00.000Z/</dc:date>      
          <echo:datasetId>LANDSAT MSS Data Received at the NASDA Station in Japan</echo:datasetId>      
          <echo:shortName>LANDSAT-MSS-NASDA</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>LANDSAT-MSS-NASDA</echo:entryId>      
          <echo:dataCenter>SCIOPS</echo:dataCenter>      
          <echo:archiveCenter>JP/RESTEC</echo:archiveCenter>      
          <echo:organization>JP/RESTEC</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>23 115 55 153</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1214584333-SCIOPS</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <title type="text">LANDSAT Thematic Mapper Data Received at the NASDA Station in Japan</title>      
          <summary type="text">The LANDSAT Thematic Mapper, a mechanical scanning radiometer, operates in 7
channels of electro-magnetic spectra, including visual, near infrared and
thermal infrared.

Data collected by the Earth Observation Center, NASDA, cover a circular area
about 5000 km diameter.  The Earth Observation Center receives TM data weekdays
and every other Saturday. It performs both radiometric and geometric correction
and distributes products in the form of magnetic tape and imagery.</summary>      
          <updated/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT-TM-NASDA" hreflang="en-US" rel="enclosure" title="LANDSAT-TM-NASDA" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1214584333-SCIOPS.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1214584333-SCIOPS</dc:identifier>      
          <dc:date>1984-03-24T00:00:00.000Z/</dc:date>      
          <echo:datasetId>LANDSAT Thematic Mapper Data Received at the NASDA Station in Japan</echo:datasetId>      
          <echo:shortName>LANDSAT-TM-NASDA</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>LANDSAT-TM-NASDA</echo:entryId>      
          <echo:dataCenter>SCIOPS</echo:dataCenter>      
          <echo:archiveCenter>JP/RESTEC</echo:archiveCenter>      
          <echo:organization>JP/RESTEC</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>23 115 55 153</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1220567985-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">LANDSAT SURFACE REFLECTANCE -L7 ETM+</title>      
          <summary type="text">Landsat satellite data have been produced, archived, and distributed by the U.S. Geological Survey (USGS) since 1972.  Users rely on these data for historical study of land surface change but shoulder the burden of post-production processing to create applications-ready data sets.</summary>      
          <updated/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT_SURFACE_REFLECTANCE_L7_ETM" hreflang="en-US" rel="enclosure" title="LANDSAT_SURFACE_REFLECTANCE_L7_ETM" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1220567985-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1220567985-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-13T00:00:00.000Z/</dc:date>      
          <echo:datasetId>LANDSAT SURFACE REFLECTANCE -L7 ETM+</echo:datasetId>      
          <echo:shortName>LANDSAT_SURFACE_REFLECTANCE_L7_ETM</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>LANDSAT_SURFACE_REFLECTANCE_L7_ETM</echo:entryId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1220567969-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">Landsat Surface Reflectance - L8 OLI/TIRS</title>      
          <summary type="text">Landsat satellite data have been produced, archived, and distributed by the U.S. Geological Survey (USGS) since 1972.  Users rely on these data for historical study of land surface change but shoulder the burden of post-production processing to create applications-ready data sets.</summary>      
          <updated/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS" hreflang="en-US" rel="enclosure" title="LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1220567969-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1220567969-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-01T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat Surface Reflectance - L8 OLI/TIRS</echo:datasetId>      
          <echo:shortName>LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS</echo:entryId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1235542031-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">Landsat 8</title>      
          <summary type="text">The Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) are onboard the Landsat 8 satellite, have acquired images of the Earth since  February 2013. The sensors collect images of the Earth with a 16-day repeat cycle, referenced to the Worldwide Reference System-2. The approximate scene size is 170 km north-south by 183 km east-west (106 mi by 114 mi).

Landsat 8 image data files consist of 11 spectral bands with a spatial resolution of 30 meters for bands 1-7 and bands 9-11; 15-meters for the panchromatic band 8. Delivered Landsat 8 Level-1 data typically include both OLI and TIRS data files; however, there may be OLI-only and/or TIRS-only scenes in the USGS archive.  A Quality Assurance (QA.tif) band is also included. This file provides bit information regarding conditions that may affect the accuracy and usability of a given pixel – clouds, water or snow, for example.</summary>      
          <updated/>      
          <link href="http://glovis.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://landsat.usgs.gov/landsat8.php" hreflang="en-US" rel="describedBy"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1235542031-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1235542031-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-11T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8</echo:datasetId>      
          <echo:shortName>Landsat_8</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>Landsat_8</echo:entryId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:processingLevelId>Not Provided</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-82.71 -180 82.74 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.prod</echo:tagKey>         
               <echo:data>&quot; &quot;</echo:data>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.provider</echo:tagKey>         
               <echo:data>&quot;USGSLSI&quot;</echo:data>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C3108204618-INPE</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <title type="text">Landsat Collection 2 - Level-2</title>      
          <summary type="text">Landsat Collection 2 Level-2 Science Products (https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products), consisting of atmospherically corrected surface reflectance (https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-reflectance) and surface temperature (https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-temperature) image data. Collection 2 Level-2 Science Products are available from August 22, 1982 to present. This dataset represents the Brazilian archive of Level-2 data from Landsat Collection 2 (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2) acquired by the Thematic Mapper (https://landsat.gsfc.nasa.gov/thematic-mapper/) onboard Landsat 4 and 5, the Enhanced Thematic Mapper (https://landsat.gsfc.nasa.gov/the-enhanced-thematic-mapper-plus-etm/) onboard Landsat 7, and Operatational Land Imager (https://landsat.gsfc.nasa.gov/satellites/landsat-8/spacecraft-instruments/operational-land-imager/) and Thermal Infrared Sensor (https://landsat.gsfc.nasa.gov/satellites/landsat-8/spacecraft-instruments/thermal-infrared-sensor/) onboard Landsat 8 and 9. Images are stored in cloud-optimized GeoTIFF (https://www.cogeo.org/) format.</summary>      
          <updated/>      
          <link href="https://fedeo-client.ceos.org?uid=landsat-2" hreflang="en-US" length="0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/landsat-2?httpAccept=application/vnd.iso.19139-2%2Bxml" hreflang="en-US" length="0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/landsat-2/api" hreflang="en-US" length="0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://data.inpe.br/bdc/stac/v1/collections/landsat-2/items" hreflang="en-US" length="0KB" rel="via"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/landsat-2/api" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C3108204618-INPE.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C3108204618-INPE</dc:identifier>      
          <dc:date>2008-01-01T00:00:00.000Z/2024-06-14T00:00:00.000Z</dc:date>      
          <echo:datasetId>Landsat Collection 2 - Level-2</echo:datasetId>      
          <echo:shortName>landsat-2</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:entryId>landsat-2_NA</echo:entryId>      
          <echo:dataCenter>INPE</echo:dataCenter>      
          <echo:organization>DOI/USGS</echo:organization>      
          <echo:organization>BR/INPE</echo:organization>      
          <echo:processingLevelId>NA</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-35.6706264 -75.6664631 6.8339121 -27.6837564</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C3108204500-INPE</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <title type="text">Landsat Collection 2 - Level-2 - Data Cube - LCF 16 days</title>      
          <summary type="text">Earth Observation Data Cube generated from Landsat Level-2 product over Brazil extension. This dataset is provided in Cloud Optimized GeoTIFF (COG) file format. The dataset is processed with 30 meters of spatial resolution, reprojected and cropped to BDC_MD grid Version 2 (BDC_MD V2), considering a temporal compositing function of 16 days using the Least Cloud Cover First (LCF) best pixel approach.</summary>      
          <updated/>      
          <link href="https://fedeo-client.ceos.org?uid=LANDSAT-16D-1" hreflang="en-US" length="0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/LANDSAT-16D-1?httpAccept=application/vnd.iso.19139-2%2Bxml" hreflang="en-US" length="0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/LANDSAT-16D-1/api" hreflang="en-US" length="0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://data.inpe.br/bdc/stac/v1/collections/LANDSAT-16D-1/items" hreflang="en-US" length="0KB" rel="via"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/LANDSAT-16D-1/api" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C3108204500-INPE.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C3108204500-INPE</dc:identifier>      
          <dc:date>2015-01-01T00:00:00.000Z/2023-12-31T00:00:00.000Z</dc:date>      
          <echo:datasetId>Landsat Collection 2 - Level-2 - Data Cube - LCF 16 days</echo:datasetId>      
          <echo:shortName>LANDSAT-16D-1</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:entryId>LANDSAT-16D-1_NA</echo:entryId>      
          <echo:dataCenter>INPE</echo:dataCenter>      
          <echo:organization>BR/INPE</echo:organization>      
          <echo:processingLevelId>NA</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-34.6755646 -75.9138367 5.9260044 -33.3435683</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1220567986-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">LANDSAT SURFACE REFLECTANCE L4-5 TM</title>      
          <summary type="text">Landsat satellite data have been produced, archived, and distributed by the U.S. Geological Survey (USGS) since 1972.  Users rely on these data for historical study of land surface change but shoulder the burden of post-production processing to create applications-ready data sets.</summary>      
          <updated/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT_SURFACE_REFLECTANCE_L4-5_TM" hreflang="en-US" rel="enclosure" title="LANDSAT_SURFACE_REFLECTANCE_L4-5_TM" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1220567986-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1220567986-USGS_LTA</dc:identifier>      
          <dc:date>1982-07-01T00:00:00.000Z/2013-01-01T23:59:59.999Z</dc:date>      
          <echo:datasetId>LANDSAT SURFACE REFLECTANCE L4-5 TM</echo:datasetId>      
          <echo:shortName>LANDSAT_SURFACE_REFLECTANCE_L4-5_TM</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>LANDSAT_SURFACE_REFLECTANCE_L4-5_TM</echo:entryId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
</feed>
```


**Step 3**  
>  With the IDN OpenSearch response, obtain the CWIC or FedEO OSDD endpoint according to CEOS-BP-001 for the collection by parsing the href attribute in `<link rel="search" type=”application/opensearchdescription+xml” >` (CEOS-BP-001). 

In [215]:
# IDN_COLLECTION = 'C1235542031-USGS_LTA'       # example from original CDA OpenSearch Client Guide - https://search.earthdata.nasa.gov/search?portal=idn&q=C1235542031-USGS_LTA (does not work).  
IDN_COLLECTION = "C2237824918-ORNL_CLOUD"
# IDN_COLLECTION = "C1532648157-ESA"

request_url = get_api_request(collection_template, {'geo:uid': IDN_COLLECTION} )
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?&uid=C2237824918-ORNL_CLOUD&clientId=ceosOpenSearchDoc'

In [216]:
response = requests.get( request_url )

root = ElementTree.fromstring(response.text)
# Extract <entry> element 
el = root.find('{http://www.w3.org/2005/Atom}entry')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:entry xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.w3.org/2005/Atom" xmlns:ns2="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:ns3="http://www.georss.org/georss">
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C2237824918-ORNL_CLOUD</ns0:id>
    <ns0:author>
      <ns0:name>CMR</ns0:name>
      <ns0:email>echodev@echo.nasa.gov</ns0:email>
    </ns0:author>
    <ns0:consortium>GEOSS</ns0:consortium>
    <ns0:consortium>EOSDIS</ns0:consortium>
    <ns0:title type="text">GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns0:title>
    <ns0:summary type="text">This dataset contains Global Ecosystem Dynamics Investigation (GEDI) Level 4A (L4A) Version 2 predictions of the aboveground biomass density (AGBD; in Mg/ha) and estimates of the prediction standard error within each sampled geolocated laser footprint. In this version, the granules are in sub-orbits. The algorithm setting group selection used for GEDI02_A Version 2 has been modified for Evergreen Broadleaf Trees in South America to reduce false positive errors resulting from the selection of waveform modes above ground elevation as the lowest mode. The footprints are located within the global latitude band observed by the International Space Station (ISS), nominally 51.6 degrees N and S and reported for the period 2019-04-18 to 2023-03-16. The GEDI instrument consists of three lasers producing a total of eight beam ground transects, which instantaneously sample eight ~25 m footprints spaced approximately every 60 m along-track. The GEDI beam transects are spaced approximately 600 m apart on the Earth's surface in the cross-track direction, for an across-track width of ~4.2 km. Footprint AGBD was derived from parametric models that relate simulated GEDI Level 2A (L2A) waveform relative height (RH) metrics to field plot estimates of AGBD. Height metrics from simulated waveforms associated with field estimates of AGBD from multiple regions and plant functional types (PFTs) were compiled to generate a calibration dataset for models representing the combinations of world regions and PFTs (i.e., deciduous broadleaf trees, evergreen broadleaf trees, evergreen needleleaf trees, deciduous needleleaf trees, and the combination of grasslands, shrubs, and woodlands). For each of the eight beams, additional data are reported with the AGBD estimates, including the associated uncertainty metrics, quality flags, model inputs, and other information about the GEDI L2A waveform for this selected algorithm setting group. Model inputs include the scaled and transformed GEDI L2A RH metrics, footprint geolocation variables and land cover input data including PFTs and the world region identifiers. Additional model outputs include the AGBD predictions for each of the six GEDI L2A algorithm setting groups with AGBD in natural and transformed units and associated prediction uncertainty for each GEDI L2A algorithm setting group. Providing these ancillary data products will allow users to evaluate and select alternative algorithm setting groups. Also provided are outputs of parameters and variables from the L4A models used to generate AGBD predictions that are required as input to the GEDI04_B algorithm to generate 1-km gridded products.</ns0:summary>
    <ns0:updated>2023-11-06T16:53:03.000Z</ns0:updated>
    <ns0:link href="https://daac.ornl.gov/daacdata/gedi/GEDI_L4A_AGB_Density_V2_1/" hreflang="en-US" rel="enclosure" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1_Fig1.png" hreflang="en-US" rel="icon" type="image/png" />
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" />
    <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?clientId=ceosOpenSearchDoc&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;versionId=2.1&amp;dataCenter=ORNL_CLOUD" hreflang="en-US" rel="search" title="Search for granules" type="application/atom+xml" />
    <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="search" title="Granule OpenSearch Descriptor Document" type="application/opensearchdescription+xml" />
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/C2237824918-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>C2237824918-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:00:00.000Z/2023-03-16T21:33:25.000Z</dc:date>
    <ns2:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns2:datasetId>
    <ns2:shortName>GEDI_L4A_AGB_Density_V2_1_2056</ns2:shortName>
    <ns2:versionId>2.1</ns2:versionId>
    <ns2:entryId>GEDI_L4A_AGB_Density_V2_1_2056_2.1</ns2:entryId>
    <ns2:dataCenter>ORNL_CLOUD</ns2:dataCenter>
    <ns2:archiveCenter>ORNL_DAAC</ns2:archiveCenter>
    <ns2:organization>ORNL_DAAC</ns2:organization>
    <ns2:processingLevelId>4</ns2:processingLevelId>
    <ns2:coordinateSystem>CARTESIAN</ns2:coordinateSystem>
    <ns2:orbitParameters />
    <ns3:box>-53 -180 54 180</ns3:box>
    <ns2:hasVariables>true</ns2:hasVariables>
    <ns2:hasFormats>true</ns2:hasFormats>
    <ns2:hasTransforms>false</ns2:hasTransforms>
    <ns2:hasCombine>false</ns2:hasCombine>
    <ns2:hasSpatialSubsetting>true</ns2:hasSpatialSubsetting>
    <ns2:hasTemporalSubsetting>true</ns2:hasTemporalSubsetting>
    <ns2:cloudHosted>true</ns2:cloudHosted>
    <ns2:tag>
      <ns2:tagKey>gov.nasa.eosdis</ns2:tagKey>
    </ns2:tag>
    <ns2:is_eosdis>true</ns2:is_eosdis>
  </ns0:entry>

```


Obtain the OSDD endpoint for the granule search for this collection by parsing the href attribute in `<link rel="search"
type="application/opensearchdescription+xml" >`

In [217]:
root = ElementTree.fromstring(response.text)
# Extract <link> element with the OSDD for the granule search
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:link xmlns:ns0="http://www.w3.org/2005/Atom" href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="search" title="Granule OpenSearch Descriptor Document" type="application/opensearchdescription+xml" />
    
```


In [218]:
# work-around
# Extract the short name for future use.  Should not be needed according to CEOS OpenSearch Best Practices.  
el = root.find('{http://www.w3.org/2005/Atom}entry/{https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom}shortName')
el.text
short_name = el.text

**Step 4**  
>  Based on the retrieved OSDD, formulate an OpenSearch request for granules belonging to that collection, directed to the relevant server.  

In [219]:
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
url_osdd = el.attrib['href']
el.attrib['href']

'https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&clientId=ceosOpenSearchDoc'

In [220]:
# Get the OSDD document
response = requests.get( el.attrib['href'] )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")


```xml
<?xml version="1.0" ?><feed xmlns="http://a9.com/-/spec/opensearch/1.1/" xmlns:eop="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/paramseters/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">  
    <title>OpenSearch Exception</title>  
    <subtitle type="text">No provider present within the `org.ceos.wgiss.cwic.granules.provider` tag.</subtitle>  
    <updated>2024-12-06T15:24:53Z</updated>  
    <id>https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc</id>  
    <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc" rel="self" title="self" type="application/atom+xml"/>  
    <totalResults>0</totalResults>  
</feed>
```


**Step 5 - work around (to correct OSDD URL for non-FedEO collections in IDN)**  
>  The OSDD URL currently returned by IDN/CMR is incorrect as reported Tuesday, April 2, 2024 at 9:58 AM.  The URL retrieved from the collection search response cannot be used.  Another one using the short_name has to be constructed.

```{warning}
The URL included in the response in case of non-FedEO collections returned by IDN is not correct and should use the `short_name`.  The following code is a "patch" for this issue.
```

In [221]:
root = ElementTree.fromstring(response.text)

try:
    el = root.find('{http://a9.com/-/spec/opensearch/1.1/}title')
    print(el.text)

    if (el.text == 'OpenSearch Exception'):
        url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=" + short_name
        print("Corrected OSDD URL: " + url_osdd)
except:
    pass

OpenSearch Exception
Corrected OSDD URL: https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056


In [222]:
# could not extract this from Atom response due to incorrect link.  Set the expected link
# The correct way to get the OSDD seems using the shortName, not collectionConceptId

# url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=Landsat_8"   # Example 1
# url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056"  # Example 2

In [223]:
url_osdd

'https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056'

In [224]:
# Get the OSDD document for granule search
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><os:OpenSearchDescription xmlns:atom="http://www.w3.org/2005/Atom" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:referrer="http://www.opensearch.org/Specifications/OpenSearch/Extensions/Referrer/1.0" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">  
    <os:ShortName>CMR Granules</os:ShortName>  
    <os:Description>NASA CMR Granule search using geo, time and parameter extensions</os:Description>  
    <os:Contact>echodev@echo.nasa.gov</os:Contact>  

    <os:Url params:method="GET" rel="results" template="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?datasetId={echo:datasetId?}&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;versionId={echo:versionId?}&amp;dataCenter={echo:dataCenter?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;parentIdentifier={eo:parentIdentifier?}" type="application/atom+xml">    
        <params:Parameter minimum="0" name="datasetId" title="Inventory associated with a dataset expressed as an ID" uiDisplay="Collection identifier" value="{echo:datasetId}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier identifier" value="{geo:uid}"/>    
        <params:Parameter minimum="0" name="parentIdentifier" title="Inventory associated with a dataset expressed as a CMR concept ID" uiDisplay="CMR Collection concept identifier" value="{eo:parentIdentifier}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  

    <os:Url params:method="GET" rel="results" template="https://cmr.earthdata.nasa.gov/opensearch/granules.html?datasetId={echo:datasetId?}&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;versionId={echo:versionId?}&amp;dataCenter={echo:dataCenter?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;parentIdentifier={eo:parentIdentifier?}" type="text/html">    
        <params:Parameter minimum="0" name="datasetId" title="Inventory associated with a dataset expressed as an ID" uiDisplay="Collection identifier" value="{echo:datasetId}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier identifier" value="{geo:uid}"/>    
        <params:Parameter minimum="0" name="parentIdentifier" title="Inventory associated with a dataset expressed as a CMR concept ID" uiDisplay="CMR Collection concept identifier" value="{eo:parentIdentifier}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  

    <os:Query echo:dataCenter="LAADS" echo:shortName="MOD02QKM" echo:versionId="005" geo:box="-180.0,-90.0,180.0,90.0" role="example" time:end="2009-05-04T00:00:00Z" time:start="2002-05-04T00:00:00Z" title="Sample search"/>  

    <os:Attribution>NASA CMR</os:Attribution>  
    <os:SyndicationRight>open</os:SyndicationRight>  
    <os:Tags>CMR NASA CWIC CEOS-OS-BP-V1.1/L3 ESIP OGC granule pageOffset=1 indexOffset=0</os:Tags>  
</os:OpenSearchDescription>
```


In [225]:
# Extract URL template for granule search
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
granule_search_template = root.find('os:Url[@rel="results"][@type="application/atom+xml"]', ns).attrib['template']

# collection_template = collection_url_atom.attrib['template']
granule_search_template

'https://cmr.earthdata.nasa.gov/opensearch/granules.atom?datasetId={echo:datasetId?}&shortName=GEDI_L4A_AGB_Density_V2_1_2056&versionId={echo:versionId?}&dataCenter={echo:dataCenter?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&parentIdentifier={eo:parentIdentifier?}'

In [226]:
request_url = get_api_request(granule_search_template, {'time:start': '1972-09-26T00:00:00Z'} )
request_url

'https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&shortName=GEDI_L4A_AGB_Density_V2_1_2056&startTime=1972-09-26T00:00:00Z'

In [227]:
response = requests.get( request_url )

# xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
# md("```xml\n" + xmlstr + "\n```\n")

root = ElementTree.fromstring(response.text)
# Extract <entry> element 
# el = root.find('{http://www.w3.org/2005/Atom}entry')
xmltxt = ElementTree.tostring(root, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:feed xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.w3.org/2005/Atom" xmlns:ns1="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:ns2="http://a9.com/-/spec/opensearch/1.1/" xmlns:ns3="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:ns4="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:ns6="http://www.georss.org/georss" ns1:version="1.2">
  <ns0:updated>2024-12-06T15:24:57.291Z</ns0:updated>
  <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom</ns0:id>
  <ns0:author>
    <ns0:name>CMR</ns0:name>
    <ns0:email>echodev@echo.nasa.gov</ns0:email>
  </ns0:author>
  <ns0:title type="text">ECHO granule metadata</ns0:title>
  <ns0:subtitle type="text">Search parameters: shortName =&gt; GEDI_L4A_AGB_Density_V2_1_2056 startTime =&gt; 1972-09-26T00:00:00Z</ns0:subtitle>
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?datasetId=GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1" hreflang="en-US" rel="up" type="application/atom+xml" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z" hreflang="en-US" rel="self" type="application/atom+xml" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z&amp;cursor=7487" hreflang="en-US" rel="last" type="application/atom+xml" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z&amp;cursor=2" hreflang="en-US" rel="next" type="application/atom+xml" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml" />
  <ns0:link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html" />
  <ns2:Query role="request" ns4:start="1972-09-26T00:00:00Z" ns3:shortName="GEDI_L4A_AGB_Density_V2_1_2056" />
  <ns2:totalResults>74860</ns2:totalResults>
  <ns2:itemsPerPage>10</ns2:itemsPerPage>
  <ns2:startIndex>1</ns2:startIndex>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613731222-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613731222-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613731222-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:03:08.000Z/2019-04-17T23:10:25.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>86.071746</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-21.8339844 90.6664734 -18.9093285 93.1515121 -15.9455862 95.5423431 -12.9488249 97.8571167 -9.9263477 100.111557 -6.8845229 102.3207092 -3.8295021 104.4995575 -0.767069 106.6615677 0.096466 107.2697449 0.125464 107.228569 -0.738065 106.6203918 -3.800411 104.4583588 -6.8552461 102.2794342 -9.8967772 100.0701675 -12.9188538 97.8155899 -15.9151096 95.5006485 -18.8782387 93.109642 -21.8022995 90.6243057 -21.8339844 90.6664734</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613707402-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613707402-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613707402-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:10:25.000Z/2019-04-17T23:33:09.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>233.259346</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>0.096889 107.2700424 3.1605549 109.4303436 6.2186122 111.6051331 9.2651892 113.8084488 12.2944918 116.0545349 15.3001385 118.3584747 18.2749271 120.7357635 21.2112904 123.2036057 24.1017456 125.7802048 26.9362698 128.4854889 29.7046547 131.3400726 32.3950768 134.3668213 34.9940529 137.589798 37.4862328 141.0342407 43.9677429 152.5696564 45.8524971 157.1061401 47.5218086 161.9548035 48.9514313 167.1147919 50.1127739 172.559967 50.9826126 178.2515564 51.5400848 -175.8731384 51.7517281 -171.0981903 51.8020554 -171.1013336 51.5901413 -175.8820496 51.0319939 178.2358551 50.1611557 172.5381622 48.998558 167.0877228 47.5674973 161.9234161 45.8966331 157.0713043 44.0101318 152.5318604 37.5243111 140.9927063 35.0307579 137.547699 32.4305153 134.3244476 29.7389412 131.2976074 26.9695206 128.4430542 24.1340752 125.7378922 21.2428226 123.1614761 18.3057709 120.6938324 15.3304052 118.3167343 12.3242941 116.0129776 9.2946329 113.7670441 6.2478008 111.5638504 3.1895959 109.3891296 0.125886 107.2288666 0.096889 107.2700424</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613709653-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613709653-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613709653-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:36:31.000Z/2019-04-17T23:56:45.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>289.630746</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>50.3259239 -151.1632233 49.2252464 -145.6598206 47.8514786 -140.4356079 46.2308769 -135.5182343 44.3904305 -130.9140472 42.3554993 -126.6189728 40.1509552 -122.6179504 37.7999954 -118.8920975 35.3218842 -115.4155655 32.7349548 -112.1641464 30.0544357 -109.1125031 27.2942162 -106.2359772 24.4663277 -103.5119019 21.5813046 -100.9187241 18.6484966 -98.4365997 15.6760359 -96.0472336 12.6721449 -93.7337112 9.6435022 -91.4798203 6.596983 -89.270813 3.5376461 -87.0910721 0.472189 -84.9280548 -0.233585 -84.43116 -0.204593 -84.3899841 0.501189 -84.886879 3.5667281 -87.049881 6.6262422 -89.2295456 9.6730461 -91.438446 12.7020788 -93.6921921 15.7064657 -96.0055466 18.6795368 -98.3947372 21.6130619 -100.8766861 24.4989128 -103.4697113 27.3277473 -106.1936874 30.0890274 -109.0702133 32.7707253 -112.121994 35.3589401 -115.3737564 37.8384399 -118.850914 40.1908798 -122.577774 42.3969688 -126.5802917 44.4334869 -130.8774719 46.2755051 -135.4844971 47.8976212 -140.4055328 49.2727737 -145.6343231 50.3745461 -151.1426544 50.3259239 -151.1632233</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613767029-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613767029-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613767029-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:56:45.000Z/2019-04-17T23:59:42.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>40.975817</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-0.23401 -84.4308624 -3.2991791 -82.268837 -6.3571992 -80.0932083 -9.2515869 -77.9995956 -9.2221441 -77.9581985 -6.328001 -80.051918 -3.2701299 -82.2276306 -0.20501 -84.3896866 -0.23401 -84.4308624</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613741178-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613741178-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613741178-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T00:41:33.000Z/2019-04-18T00:43:06.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>17.02199</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-4.6459689 80.3302689 -1.585824 82.4953766 0.09879 83.6814117 0.127777 83.6402283 -1.556802 82.4542084 -4.6168618 80.289032 -4.6459689 80.3302689</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613728635-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613728635-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613728635-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T00:43:06.000Z/2019-04-18T01:06:16.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>356.752358</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>0.099208 83.6817093 3.1628549 85.8422318 6.2209601 88.0170898 9.2673845 90.2206497 12.2967377 92.4668427 15.3021154 94.7709198 18.2766266 97.1482544 21.2129631 99.6162949 24.1030331 102.1928482 26.9373379 104.8982239 29.7057495 107.7529068 32.3960228 110.7800674 34.9951553 114.0032806 37.4867973 117.4478683 39.8553276 121.1404724 42.0786362 125.1035538 44.1362953 129.3586578 46.0028419 133.9211731 47.6527596 138.797226 49.0604744 143.9837189 50.1977348 149.4519958 51.0410347 155.1615295 51.5710526 161.0489502 51.7738762 167.0356445 51.7756119 168.0159607 51.8259735 168.0162811 51.824234 167.0345459 51.6211472 161.0406342 51.0904884 155.1463776 50.2462158 149.4306793 49.1077156 143.957077 47.6985779 138.7661743 46.0471153 133.8866119 44.1789742 129.3213959 42.1197357 125.0643234 39.8948784 121.0998459 37.5248833 117.4063263 35.0318604 113.9611816 32.4314651 110.7376862 29.740036 107.7104263 26.9705925 104.8557968 24.1353664 102.1505432 21.2444973 99.574173 18.3074703 97.1063232 15.3323851 94.729187 12.3265409 92.425293 9.2968321 90.1792526 6.2501502 87.9758072 3.1918981 85.8010254 0.128207 83.6405334 0.099208 83.6817093</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613738123-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613738123-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613738123-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T01:06:16.000Z/2019-04-18T01:08:30.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>27.728352</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>51.7756081 168.0166779 51.5928535 174.006897 51.0830269 179.9024353 50.9306984 -178.8402405 50.9800682 -178.8244476 51.1325951 179.916626 51.643013 174.0141907 51.8259697 168.017334 51.7756081 168.0166779</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613712163-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613712163-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613712163-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T01:37:02.000Z/2019-04-18T01:45:13.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>96.386522</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-23.0758801 -90.5495224 -25.9292355 -87.8976212 -28.719553 -85.1057281 -31.4343376 -82.1522675 -34.066246 -79.0105362 -36.5951424 -75.659523 -39.0072632 -72.0722198 -41.2829971 -68.2245407 -43.4022217 -64.0936508 -43.7361984 -63.3814812 -43.6937943 -63.3438873 -43.3601494 -64.0555573 -41.2424965 -68.1847229 -38.9682808 -72.0312042 -36.5575867 -75.6177444 -34.0300255 -78.9683075 -31.399332 -82.1098328 -28.6856613 -85.0632553 -25.8963356 -87.8552322 -23.0437202 -90.5074005 -23.0758801 -90.5495224</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613707648-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613707648-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613707648-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T02:02:47.000Z/2019-04-18T02:11:37.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>113.29673</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-37.3154259 26.5281448 -34.8186836 29.9455051 -32.2175102 33.1447144 -29.5261726 36.1499557 -26.7580585 38.9866676 -23.9241676 41.6754875 -21.0334892 44.2404213 -18.0979824 46.6972198 -15.1244192 49.0658646 -12.6770945 50.943119 -12.6471701 50.9016037 -15.0940886 49.0242195 -18.0670738 46.6553917 -21.0018826 44.1984177 -23.891758 41.6333237 -26.72472 38.9444008 -29.4918041 36.1076546 -32.1819878 33.1025238 -34.7819061 29.9036064 -37.277462 26.4865437 -37.3154259 26.5281448</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613752613-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2023-11-06T16:53:18.000Z</ns0:updated>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5.sha256" hreflang="en-US" rel="enclosure" title="Download GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5.sha256" />
    <ns0:link href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" type="text/html" />
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" type="application/pdf" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" />
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" hreflang="en-US" rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" />
    <ns0:link href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" />
    <ns0:link href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" hreflang="en-US" rel="describedBy" type="text/html" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://doi.org/10.3334/ORNLDAAC/2056" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://opendap.earthdata.nasa.gov/" hreflang="en-US" rel="http://esipfed.org/ns/fedsearch/1.1/service#" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" hreflang="en-US" rel="describedBy" type="application/pdf" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" hreflang="en-US" rel="describedBy" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://gedi.umd.edu" hreflang="en-US" rel="via" ns3:inherited="true">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613752613-ORNL_CLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>G2613752613-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T02:15:57.000Z/2019-04-18T02:38:57.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>324.322707</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>0.579059 60.4911842 3.642535 62.6533508 6.6990881 64.8318405 9.7432947 67.0409622 12.7691469 69.2953491 15.7702131 71.6097717 18.7392311 74.0002365 21.6690865 76.483963 24.5510254 79.0796356 27.376009 81.8069077 30.1319752 84.686348 32.8091354 87.7424774 35.3923035 90.9988632 37.8662415 94.4803543 40.213459 98.2141571 42.4124565 102.2218399 44.4413834 106.5232315 46.2767677 111.1362915 47.8905945 116.0625076 49.2572479 121.2932892 50.3497734 126.8029556 51.1454773 132.5445099 51.6247368 138.4529724 51.7762222 144.4246979 51.8265839 144.4247131 51.6749001 138.4457703 51.1950531 132.5303955 50.3984261 126.7825546 49.3046951 121.2674103 47.9366417 116.0320816 46.3212852 111.102211 44.484314 106.4863358 42.4537964 102.1828766 40.2532463 98.1737137 37.9045448 94.4389267 35.4292183 90.9568329 32.8447685 87.700119 30.1664371 84.6438751 27.4094162 81.7644653 24.5834961 79.0373077 21.7007351 76.441803 18.7701778 73.9582672 15.8005648 71.5680084 12.7990179 69.2537689 9.772789 66.9995422 6.728313 64.7905426 3.6715939 62.612133 0.608061 60.4500084 0.579059 60.4911842</ns6:polygon>
  </ns0:entry>
</ns0:feed>
```


## Obtaining the OpenSearch Description Document (OSDD)

OpenSearch Description Documents (OSDDs) provide necessary information for clients to programmatically formulate valid search requests. Specifically, clients are expected to acquire both the cardinality and the domain of request parameters based on the query template in the OSDD. Collection valids (e.g. spatial bounding coordinates and temporal extent) are also provided in the OSDD in both machine parsable and human readable formats. Collection valids enable clients to formulate valid requests yielding more accurate results. 

The federated system provides both generic and collection specific OSDDs.  IDN provides the generic OSDD to be used for collection search.  Alternatively, clients are also able to fetch a generic OSDD through the CWIC or FedEO OSDD endpoint.  The OSDD request should also include a client identifier string, as recommended by the CEOS OpenSearch Best Practices.

Collection-specific OSDD are returned by IDN as part of collection search results. In a collection specific OSDD, the domain is provided for some parameters (e.g., timeStart and timeEnd) in addition to the request parameter syntax. 

- Generic OSDD request URL example (CMR):  https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=foo  
- Generic OSDD request URL example (FedEO): https://fedeo.ceos.org/opensearch/description.xml 

## Search request

CEOS OpenSearch supports both searching for collections through the IDN and
for granules in a specific collection at one of the data partners (i.e. “CWIC” and ["FedEO"](chapter-fedeo)). It executes a collection or inventory search (a.k.a. granule search), as appropriate, and returns the matching results.

In order to initialize a valid request, clients will fill request parameters with proper values. The
template of the OpenSearch request is available under the `<Url>` element in the OSDD.

## Search response

OpenSearch does not define or require any specific encoding format for the search response. 

Instead, it defines a set of search-related metadata elements which can be inserted into existing encoding formats. Typically, list-based XML syndication formats - such as RSS 2.0 and [Atom 1.0](https://datatracker.ietf.org/doc/html/rfc4287) - are used. 

The OpenSearch response elements include: 

- `totalResults`: number of total results 
- `startIndex`: index number corresponding to the first entry item returned 
- `itemsPerPage`: number of results returned 
- `Query`: search query to get the same search response 

Below is an example of a search response from the IDN in Atom, to the free keyword search “water”. 

In [228]:
request_url = get_api_request(collection_template, {'count': '1', 'searchTerms': 'water', 'startIndex': '1', 'startPage': '1'} )
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=water&cursor=1&numberOfResults=1&offset=1&clientId=ceosOpenSearchDoc'

In [229]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-12-06T15:24:59.526Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; water</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=water&amp;cursor=1&amp;numberOfResults=1&amp;offset=1&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=water&amp;cursor=1&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;offset=23204" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=water&amp;cursor=1&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;offset=0" hreflang="en-US" rel="previous" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=water&amp;cursor=1&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;offset=2" hreflang="en-US" rel="next" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=water&amp;cursor=1&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;offset=0" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query os:searchTerms="water" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>23204</os:totalResults>   
     <os:itemsPerPage>1</os:itemsPerPage>   
     <os:startIndex>2</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C2799438266-POCLOUD</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>GEOSS</consortium>      
          <consortium>EOSDIS</consortium>      
          <title type="text">SWOT Level 2 Water Mask Pixel Cloud Data Product, Version C</title>      
          <summary type="text">Point cloud of water mask pixels (“pixel cloud”) with geolocated heights, backscatter, geophysical fields, and flags. Point cloud over tile (approx 64x64 km2); half swath (left or right side of full swath). Available in netCDF-4 file format.&lt;br&gt;
Please note that this collection contains SWOT Version C science data products.</summary>      
          <updated>2022-07-20T18:09:33.000Z</updated>      
          <link href="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-docs/web-misc/swot_mission_docs/D-109532_SWOT_UserHandbook_20240502.pdf" hreflang="en-US" rel="describedBy" type="application/pdf"/>      
          <link href="https://swot.jpl.nasa.gov" hreflang="en-US" rel="via"/>      
          <link href="https://swot.jpl.nasa.gov" hreflang="en-US" rel="describedBy"/>      
          <link href="https://www.nasa.gov/swot" hreflang="en-US" rel="describedBy"/>      
          <link href="https://www.jpl.nasa.gov/missions/surface-water-and-ocean-topography-swot" hreflang="en-US" rel="describedBy"/>      
          <link href="https://podaac.jpl.nasa.gov/swot" hreflang="en-US" rel="describedBy"/>      
          <link href="https://www.aviso.altimetry.fr/en/missions/current-missions/swot.html" hreflang="en-US" rel="describedBy" type="text/html"/>      
          <link href="https://github.com/podaac/data-subscriber" hreflang="en-US" rel="via"/>      
          <link href="https://podaac.jpl.nasa.gov/CitingPODAAC" hreflang="en-US" rel="describedBy"/>      
          <link href="https://cmr.earthdata.nasa.gov/virtual-directory/collections/C2799438266-POCLOUD" hreflang="en-US" length="0.0MB" rel="enclosure"/>      
          <link href="https://search.earthdata.nasa.gov/search/granules?p=C2799438266-POCLOUD" hreflang="en-US" length="0.0MB" rel="enclosure"/>      
          <link href="https://podaac.jpl.nasa.gov/Podaac/thumbnails/SWOT_L2_HR_PIXC_2.0.jpg" hreflang="en-US" rel="icon" type="image/jpeg"/>      
          <link href="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-docs/web-misc/swot_mission_docs/pdd/D-56411_SWOT_Product_Description_L2_HR_PIXC_20240913.pdf" hreflang="en-US" rel="describedBy" type="application/pdf"/>      
          <link href="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-docs/web-misc/swot_mission_docs/atbd/D-105504_SWOT_ATBD_L2_HR_PIXC_20230713a_cite2.pdf" hreflang="en-US" rel="describedBy" type="application/pdf"/>      
          <link href="https://search.earthdata.nasa.gov/search/granules?p=C2799438266-POCLOUD&amp;pg[0][id]=*_PIC*" hreflang="en-US" rel="enclosure"/>      
          <link href="https://search.earthdata.nasa.gov/search/granules?p=C2799438266-POCLOUD&amp;pg[0][id]=*PGC0*" hreflang="en-US" rel="enclosure"/>      
          <link href="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-docs/web-misc/swot_mission_docs/releases/SWOT_VersionC_KaRIn_Products_Release_Note_20241014.pdf" hreflang="en-US" rel="describedBy" type="application/pdf"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?clientId=ceosOpenSearchDoc&amp;shortName=SWOT_L2_HR_PIXC_2.0&amp;versionId=2.0&amp;dataCenter=POCLOUD" hreflang="en-US" rel="search" title="Search for granules" type="application/atom+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2799438266-POCLOUD&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="search" title="Granule OpenSearch Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C2799438266-POCLOUD.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C2799438266-POCLOUD</dc:identifier>      
          <dc:date>2022-12-16T00:00:00.000Z/</dc:date>      
          <echo:datasetId>SWOT Level 2 Water Mask Pixel Cloud Data Product, Version C</echo:datasetId>      
          <echo:shortName>SWOT_L2_HR_PIXC_2.0</echo:shortName>      
          <echo:versionId>2.0</echo:versionId>      
          <echo:entryId>SWOT_L2_HR_PIXC_2.0_2.0</echo:entryId>      
          <echo:dataCenter>POCLOUD</echo:dataCenter>      
          <echo:archiveCenter>NASA/JPL/PODAAC</echo:archiveCenter>      
          <echo:organization>NASA/JPL/PODAAC</echo:organization>      
          <echo:organization>Jet Propulsion Laboratory</echo:organization>      
          <echo:processingLevelId>2</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters inclinationAngle="77.6" numberOfOrbits="0.5" period="102.89" swathWidth="120.0"/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>true</echo:hasVariables>      
          <echo:hasFormats>true</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>true</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>true</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>true</echo:cloudHosted>      
          <relevance:score>1.43</relevance:score>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
</feed>
```


Notice the OpenSearch elements which appear inside the feed element. Namespaces referred in the OpenSearch response are listed as follows: 

In [230]:
# Get namespace declarations from root element (all atributes)
n = minidom.parseString(response.text).documentElement
attrs = dict(n.attributes.items())
attrs.pop('esipdiscovery:version') # present in IDN but not a namespace

# Sorting and printing key-value pairs by the key
for i in sorted(attrs):
    print('{:25}'.format(i) + "    " + attrs[i])

xmlns                        http://www.w3.org/2005/Atom
xmlns:dc                     http://purl.org/dc/elements/1.1/
xmlns:echo                   https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom
xmlns:eo                     http://a9.com/-/opensearch/extensions/eo/1.0/
xmlns:esipdiscovery          http://commons.esipfed.org/ns/discovery/1.2/
xmlns:georss                 http://www.georss.org/georss
xmlns:gml                    http://www.opengis.net/gml
xmlns:os                     http://a9.com/-/spec/opensearch/1.1/
xmlns:relevance              http://a9.com/-/opensearch/extensions/relevance/1.0/
xmlns:time                   http://a9.com/-/opensearch/extensions/time/1.0/


**Atom <feed> element, Granule Search**

In [231]:
%%html

<table>
<tr><th>Element</th><th align="left">Value</th></tr>

<tr><td>atom: title</td><td align="left">Atom feed title element</td></tr>
<tr><td>atom: updated</td><td align="left">Date tag indicating when granule metadata is returned from data provider</td></tr>
<tr><td>atom: author</td><td align="left">Fixed value, which is the contact information of supporting team</td></tr>

<tr><td>atom: id</td><td align="left">Fixed value for search identifier</td></tr>
<tr><td>opensearch: totalResults</td><td align="left"> Number of records matched</td></tr>
<tr><td>opensearch: startIndex</td><td align="left"> Number of start record desired by client</td></tr>
<tr><td>opensearch: itemsPerPage</td><td align="left"> Actual number of returned items per page</td></tr>
<tr><td>opensearch: Query</td><td align="left">Query element recording actual request parameter values from client</td></tr>
<tr align="left"><td>atom: link</td><td align="left"> Traversal link. Supported ‘rel’ attribute values include:
    
<ul><li>first: link to the first matching result</li>
<li>last: link to the last matching result</li>
<li>previous: link to previous granule, where applicable</li>
<li>next: link to next granule, where applicable</li>
<li>profile: link to specified profile(s)</li>
<li>up: link to parent identifier</li>
<li>search: link to OSDD endpoint</li>
<li>self: link of submitted OpenSearch request</li>
</ul></td></tr>
</table>

Element,Value
atom: title,Atom feed title element
atom: updated,Date tag indicating when granule metadata is returned from data provider
atom: author,"Fixed value, which is the contact information of supporting team"
atom: id,Fixed value for search identifier
opensearch: totalResults,Number of records matched
opensearch: startIndex,Number of start record desired by client
opensearch: itemsPerPage,Actual number of returned items per page
opensearch: Query,Query element recording actual request parameter values from client
atom: link,"Traversal link. Supported ‘rel’ attribute values include: first: link to the first matching result last: link to the last matching result previous: link to previous granule, where applicable next: link to next granule, where applicable profile: link to specified profile(s) up: link to parent identifier search: link to OSDD endpoint self: link of submitted OpenSearch request"


**Atom <entry> element**

In [232]:
%%html

<table width="50%">
<tr><th>Element</th><th>Value</th></tr>

<tr><td>atom:title</td><td align="left">Descriptive title for the entry</td></tr>
<tr><td>atom:id</td><td align="left">Unique identifier of the entry within the local catalog system</td></tr>
<tr><td>atom: updated</td><td align="left">Date tag indicating when entry metadata is last updated by data provider</td></tr>

<tr><td>atom:author</td><td align="left"> Fixed value, which is the contact information of data provider</td></tr>
<tr><td>spatial extent elements</td><td align="left">For each granule, at least one &lt;georss:box&gt; will be provided to represent the
minimum bounding rectangle of spatial extent of entry, if applicable.
&lt;georss:box&gt; is formatted with coordinate order of WestBoundingLongitude, SouthBoundingLatitude, EastBoundingLongitude, NorthBoundingLatitude. All coordinates are in EPSG:4326
&ltgeorss:polygon&gt; and other geometries may also be provided if it is available in data provider’s metadata.

<tr><td>temporal element</td><td align="left">For each entry, a single <dc:date> element will be provided to represent the
temporal extent of the entry, if applicable.
e.g. 1989 - 10 - 19T00:00:00.000Z/1989- 10 - 21T23:59:59.000Z</td></tr>
<tr><td>atom:link</td><td align="left">Supported values under ‘rel’ attribute: via/enclosure/alternate/icon
(See Table below)</td></tr>
<tr><td>atom:summary</td><td align="left">Summary descriptive text for the entry</td></tr>

</table>

**‘rel’ attributes of Atom <link> element**

In [234]:
%%html

<table>
<tr><th>‘rel’ attribute</th><th>Artifact</th></tr>

<tr><td>via</td><td align="left">Metadata</td></tr>
<tr><td>icon</td><td align="left">Browse</td></tr>
<tr><td>alternate</td><td align="left">Documentation</td></tr>
<tr><td>describedby</td><td align="left">Documentation</td></tr>
<tr><td>enclure</td><td align="left">Product</td></tr>
<tr><td>up</td><td align="left">Collection Metadata</td></tr>
</table>

‘rel’ attribute,Artifact
via,Metadata
icon,Browse
alternate,Documentation
describedby,Documentation
enclure,Product
up,Collection Metadata


## Error Handling

The CEOS OpenSearch Best Practices (CEOS-BP-017) recommends the use of HTTP status codes
as following, 4xx for client errors, and 5xx for server errors.

- 400 Bad Request: The request has an invalid syntax (i.e. badly formatted geometry)
- 413 Request Entity Too Large: The request originates too many returnable hits
- 415 Unsupported media type: Media type in the request is not available or valid.
- 500 Internal Server Error: Default code for the server side for an execution error.
- 501 Not Implemented: When requesting an unimplemented feature (e.g. relation operator not supported).
- 503 Service Unavailable: When the search service is temporarily not available (due to overload or other reasons).
- 504 Gateway Timeout: When the search engine is a broker or aggregator to other services that fail to produce an answer within a giving time frame.

CEOS OpenSearch implementations are recommended to support these codes.